In [18]:
import dask.dataframe as dd

In [19]:
ddf = dd.read_parquet("../data/parking_tickets/parquet/full_data_cleaned.parquet")
ddf

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Latitude,Longitude
npartitions=20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,string,string,string,int64,int64,string,string,string,int64,int64,int64,int64,int64,int64,int64,int64,string,string,string,string,string,string,string,string,int64,int64,string,string,string,string,string,string,int64,int64,string,string,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [20]:
# get the top 10 streets with the most tickets
top_streets = ddf.groupby("Street Code1").size().nlargest(10)
top_streets.compute()

Street Code1
13610    1642187
10210    1276505
25390     932468
10110     814370
24890     790814
10010     696974
59990     617707
10410     617646
10810     565291
10510     541612
dtype: int64

In [21]:
# get the corresponding names
top_streets_names = ddf[ddf["Street Code1"].isin(top_streets.index)][["Street Code1", "Street Name"]].drop_duplicates()
top_streets_names = top_streets_names.compute()
top_streets_names

,Street Code1,Street Name
8328,10210,3 AVE
8759,10210,3RD AV
9210,10810,8TH AVE
9396,25390,MADISON AVE
9432,10510,6TH AVENUE
...,...,...
8206044,10010,R/O 422 1ST AVE
9477367,13610,BEHIND 95 BROADWAY
9492487,10010,E/O 462 1ST AVE
9504739,13610,220th & Broadway


In [22]:
# gather all of the street names corresponding to the street code
codes_names = top_streets_names.groupby("Street Code1")["Street Name"].apply(lambda x: "||".join(x)).reset_index()
codes_names

,Street Code1,Street Name
0,10010,R/O 462 1ST AVE NY N||S/O 462 1ST AVENUE||FRON...
1,10110,2nd Ave||2ND AVE||262 2ND AVE||F/O 2244 2ND...
2,10210,3 AVE||3RD AV||3 AV||3RD AVENUE||3RD AVE||R/...
3,10410,5 AVE||FIFTH AVENUE||E/O 2285 5TH AVE||5 AVE|...
4,10510,6TH AVENUE||6 AVENUE||6 AV||AVE OF AMERICA||AV...
5,10810,8TH AVE||8 AVENUE||F/O 224 8TH AVE||C/O 8TH A...
6,13610,F/O 1450 BROADWAY||W 133 BROADWAY||R/O 3333 BR...
7,24890,LEXINGTON AVENUE||LEX||LEX. AVE||LEX AV||LEX ...
8,25390,MADISON AVE||MADISON AVENUE||191 STREET||R/O 8...
9,59990,S/O 108-18 QUEENS BL||S/O 97-77 QUEENS BLV||R/...


In [23]:
list(codes_names["Street Code1"])

[10010, 10110, 10210, 10410, 10510, 10810, 13610, 24890, 25390, 59990]